In [2]:
# создаем debezium коннектор
import requests
import json

url = "http://kafka-connect:8083/connectors"

payload = {
    "name": "postgres-backend-debezium-connector",
    "config": {
        "connector.class": "io.debezium.connector.postgresql.PostgresConnector",
        "plugin.name": "pgoutput",
        "database.hostname": "postgres",
        "database.port": "5432",
        "database.user": "airflow",
        "database.password": "airflow",
        "database.dbname": "backend",
        "topic.prefix": "pg_dev",
        "table.include.list": "public.*",
        "heartbeat.interval.ms": "5000",
        "slot.name": "dbname_debezium",
        "publication.name": "dbname_publication",

        "transforms": "AddPrefix",
        "transforms.AddPrefix.type": "org.apache.kafka.connect.transforms.RegexRouter",
        "transforms.AddPrefix.regex": "pg_dev\\.public\\.(.*)",
        "transforms.AddPrefix.replacement": "data.cdc.dbname_$1",

        "key.converter": "org.apache.kafka.connect.json.JsonConverter",
        "value.converter": "org.apache.kafka.connect.json.JsonConverter",
        "key.converter.schemas.enable": "false",
        "value.converter.schemas.enable": "false",

        "include.schema.changes": "false",
        "snapshot.mode": "initial"
    }
}

headers = {
    "Content-Type": "application/json"
}

try:
    response = requests.post(url, headers=headers, data=json.dumps(payload), timeout=10)
    print("Status:", response.status_code)
    print("Response:")
    print(response.json())

except requests.exceptions.RequestException as e:
    print("Error:", e)

Status: 409
Response:
{'error_code': 409, 'message': 'Connector postgres-backend-debezium-connector already exists'}


In [3]:
# Проверяем статус коннектора
import requests

url = "http://kafka-connect:8083/connectors/postgres-backend-debezium-connector/status"

try:
    response = requests.get(url, timeout=10)
    response.raise_for_status()  # выбросит ошибку, если HTTP-код != 200
    
    print("Status code:", response.status_code)
    print("Response JSON:")
    print(response.json())  # печатаем JSON красиво

except requests.exceptions.RequestException as e:
    print("Error while sending request:", e)


Status code: 200
Response JSON:
{'name': 'postgres-backend-debezium-connector', 'connector': {'state': 'RUNNING', 'worker_id': '172.18.0.11:8083'}, 'tasks': [{'id': 0, 'state': 'RUNNING', 'worker_id': '172.18.0.11:8083'}], 'type': 'source'}
